In [1]:
import playsound
import json
import data
import utils
import torch
import torch.nn as nn
import torchvision
import torchvision.models.segmentation.deeplabv3 as dlv3
import matplotlib.pyplot as plt
import numpy as np
import argparse
from torch.utils.data import DataLoader
from torch.utils.data import Subset
from torchsummary import summary
from tqdm import tqdm

In [ ]:
def validate_epoch(model, val_dataloader, loss_function):
    model.eval()
    batch_losses = []
    metrics = {'miou': 0, 'loss': 0}
    
    with torch.no_grad():
        for (inputs, targets) in tqdm(val_dataloader, desc=f'Validation batch progress'):
            ipts = torch.autograd.Variable(inputs).cuda()
            tgts = torch.autograd.Variable(targets).cuda()
            
            preds = model(ipts)['out']
            loss = loss_function(preds, tgts.squeeze(1).long())
            loss = loss.data.cpu().numpy()
            batch_losses += [loss]

            preds = torch.argmax(preds.cpu(), dim=1)
            tgts = torch.squeeze(targets, dim=1)

            metrics['miou'] += sum(utils.iou(preds, tgts, num_classes=20)) / 20
            metrics['loss'] += loss

        metrics['miou'] /= float(len(val_dataloader))
        metrics['loss'] /= float(len(val_dataloader))
    return metrics

In [ ]:
def train_epoch(model, train_dataloader, loss_function, optimizer):
    model.train()
    batch_losses = []
    
    for (inputs, targets) in tqdm(train_dataloader, desc=f'Training batch progress'):
        ipts = torch.autograd.Variable(inputs).cuda()
        tgts = torch.autograd.Variable(targets).cuda()
        pred = model(ipts)['out']

        loss = loss_function(pred, tgts.squeeze(1).long())
        loss_val = loss.data.cpu().numpy()
        batch_losses += [loss_val]

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return batch_losses

In [ ]:
def train(model, train_dataloader, val_dataloader, epochs, loss_function, optimizer, lr_decay):
    model.train()
    epoch_train_losses = []
    epoch_val_losses = []
    epoch_val_mious = []
    for epoch in tqdm(range(epochs), desc='Epoch progress'):
        batch_train_losses = train_epoch(
            model=model, 
            train_dataloader=train_dataloader,
            loss_function=loss_function, 
            optimizer=optimizer)
        
        batch_val_metrics = validate_epoch(
            model=model,
            val_dataloader=val_dataloader,
            loss_function=loss_function)

        epoch_average_train_loss = np.mean(batch_train_losses)
        epoch_train_losses += [epoch_average_train_loss]
        epoch_val_losses += [batch_val_metrics['loss']]
        epoch_val_mious += [batch_val_metrics['miou']]

        print(f'\n\n[TRAIN] Epoch average loss: {epoch_average_train_loss:.2f}')
        print(f'[VAL] Epoch average loss: {batch_val_metrics["loss"]:.2f}')
        print(f'[VAL] Epoch average miou: {batch_val_metrics["miou"]:.2f}\n')
        
        if lr_decay:
            for param_group in optimizer.param_groups:
                param_group['lr'] *= ((1.0 + float(epoch) / epochs) ** 0.9)

    plt.plot(epoch_train_losses, label='Train loss', color='blue') 
    plt.plot(epoch_val_losses, label='Validation loss', color='yellow') 
    plt.xlabel('Epoch') 
    plt.ylabel('Loss') 
    plt.title('Loss over Epochs') 
    plt.legend()
    plt.show()
 
    plt.plot(epoch_val_mious, label='Validation mIoU', color='green') 
    plt.xlabel('Epoch') 
    plt.ylabel('mIoU') 
    plt.title('mIoU over Epochs') 
    plt.legend()
    plt.show()

In [4]:
import pprint
args = {}
with open('./configs/config-1.json') as json_file:
    args = json.load(json_file)
pprint.pprint(args)

{'data-batch-size': 4,
 'data-source': 'synthetic',
 'data-subset-size': 2000,
 'epochs': 16,
 'fine-tune': False,
 'fine-tune-path': './params/deeplabv3_model.pt',
 'ignore-label': 19,
 'image-height': 256,
 'image-width': 512,
 'learning-rate': 1e-05,
 'learning-rate-paper-decay': True,
 'model-save-path': './params/deeplabv3_model.pt',
 'val-data-batch-size': 4,
 'val-data-subset-size': 200}


In [ ]:
torch.cuda.empty_cache()
train_dataset = data.HybridDataset(root_path=f'C:\\Users\\Manuel\\Projects\\GitHub_Repositories\\master_thesis\\datasets\\{args["data_source"]}\\train',
                                   input_dir='rgb',
                                   target_dir='semantic_segmentation_mapped',
                                   transform=torchvision.transforms.Compose([torchvision.transforms.Resize((args["image_height"], args["image_width"]))]),
                                   type=args["data_source"],
                                   labels_mapping=None)
train_dataset = Subset(train_dataset, indices=range(args["data_subset_size"]))
train_dataloader = DataLoader(dataset=train_dataset,
                              batch_size=args["data_batch_size"],
                              shuffle=True,
                              num_workers=1)
val_dataset = data.HybridDataset(root_path=f'C:\\Users\\Manuel\\Projects\\GitHub_Repositories\\master_thesis\\datasets\\{args["data_source"]}\\val',
                                 input_dir='rgb',
                                 target_dir='semantic_segmentation_mapped',
                                 transform=torchvision.transforms.Compose([torchvision.transforms.Resize((args["image_height"], args["image_width"]))]),
                                 type=args["data_source"],
                                 labels_mapping=None)
val_dataset = Subset(val_dataset, indices=range(args["val_data_subset_size"]))
val_dataloader = DataLoader(dataset=val_dataset,
                            batch_size=args["val_data_batch_size"],
                            shuffle=True,
                            num_workers=1)
    

In [ ]:
model = dlv3.deeplabv3_resnet50(pretrained=False, progress=True, output_stride=256, num_classes=len(data.SemanticLabelMapper.ID_TO_STRING['common'].keys()))  
if args["fine_tune"]:
    model.load_state_dict(torch.load(args["fine_tune_model_path"]))
    for name, param in model.backbone.named_parameters():
        if 'layer1' in name or 'layer2' in name or 'layer3' in name or 'layer4' in name or 'layer5' in name:
            print(f'---> Freezing layer: {name}.')
            param.requires_grad = False

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
params = utils.add_weight_decay(model, l2_value=0.0001)
optimizer = torch.optim.Adam(params=params, lr=args["learning_rate"])
loss_function = nn.CrossEntropyLoss(ignore_index=args["ignore_label"])

In [ ]:
train(model=model, 
      train_dataloader=train_dataloader, 
      val_dataloader=val_dataloader, 
      epochs=args["epochs"], 
      loss_function=loss_function, 
      optimizer=optimizer, 
      lr_decay=args["learning_rate_paper_decay"])

In [ ]:
torch.save(model.state_dict(), f'{args["model_save_path"]}',)